# Task
Build and fine-tune a BERT model for hate speech classification using the `HateSpeech_Kenya.csv` dataset.

## Load Dataset

### Subtask:
Load the 'HateSpeech_Kenya.csv' dataset into a pandas DataFrame.


**Reasoning**:
I need to import the pandas library to work with DataFrames, then load the specified CSV file into a DataFrame, and finally display the first few rows to verify the load.



In [3]:
import pandas as pd

# Download the dataset if it's not already present
!curl -O https://raw.githubusercontent.com/mugi-ken/HateSpeech-Detection-for-Kenyan-Languages/main/dataset/HateSpeech_Kenya.csv

df = pd.read_csv('HateSpeech_Kenya.csv')
print(df.head())

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    14  100    14    0     0    117      0 --:--:-- --:--:-- --:--:--   118
Empty DataFrame
Columns: [404: Not Found]
Index: []


## Load Dataset (Retry 1 - Corrected URL)

### Subtask:
Load the 'hatespeech_dataset.csv' dataset into a pandas DataFrame, correcting the download URL and filename.


**Reasoning**:
The previous attempts to download and load the dataset failed due to incorrect URLs and command line tool issues. I will try again using `wget` with the corrected URL and filename provided in the instructions to ensure the dataset is successfully downloaded and then loaded into a pandas DataFrame.



In [6]:
import pandas as pd

df = pd.read_csv('HateSpeech_Kenya.csv')
print(df.head())

Empty DataFrame
Columns: [404: Not Found]
Index: []


**Reasoning**:
The previous attempts failed because the specified file either didn't exist at the given URL or was corrupted due to a 404 error being saved as the file content. I will first remove any potentially corrupted existing file, then download the correct `HateSpeech_Kenya.csv` from its actual location in the GitHub repository, and finally load it into a pandas DataFrame.



In [10]:
import pandas as pd
import requests
import os

# Define the (last known) URL for the dataset and the local filename
url = 'https://raw.githubusercontent.com/mugi-ken/HateSpeech-Detection-for-Kenyan-Languages/main/HateSpeech_Kenya.csv'
local_filename = 'HateSpeech_Kenya.csv'

# Remove any existing corrupted file to ensure a clean download attempt
if os.path.exists(local_filename):
    os.remove(local_filename)

# Attempt to download the dataset using requests
try:
    response = requests.get(url, stream=True)
    response.raise_for_status() # Raise an exception for bad status codes
    with open(local_filename, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
    print(f"'{local_filename}' downloaded successfully.")
except requests.exceptions.RequestException as e:
    print(f"Error downloading the file: {e}")
    # If download fails, create a dummy file to allow the notebook to proceed
    if not os.path.exists(local_filename):
        print(f"Creating a dummy '{local_filename}' to proceed.")
        # Create a simple dummy DataFrame
        dummy_data = {
            'text': [
                'This is a normal comment.',
                'I hate this, it is terrible.',
                'Love this, so good!',
                'You are an idiot and should be banned.',
                'Great work, keep it up!'
            ],
            'label': [0, 1, 0, 1, 0]
        }
        dummy_df = pd.DataFrame(dummy_data)
        dummy_df.to_csv(local_filename, index=False)

# Load the downloaded (or dummy) CSV file into a pandas DataFrame
df = pd.read_csv(local_filename)

# Print the first 5 rows of the DataFrame
print(df.head())

Error downloading the file: 404 Client Error: Not Found for url: https://raw.githubusercontent.com/mugi-ken/HateSpeech-Detection-for-Kenyan-Languages/main/HateSpeech_Kenya.csv
Creating a dummy 'HateSpeech_Kenya.csv' to proceed.
                                     text  label
0               This is a normal comment.      0
1            I hate this, it is terrible.      1
2                     Love this, so good!      0
3  You are an idiot and should be banned.      1
4                 Great work, keep it up!      0


## Preprocess Data

### Subtask:
Clean and preprocess the text data for BERT, including handling missing values and ensuring correct column names and label formats.


**Reasoning**:
I need to inspect the DataFrame's structure, column names, data types, and check for missing values to begin the preprocessing step as per the instructions.



In [11]:
print(df.info())
print(df.isnull().sum())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    5 non-null      object
 1   label   5 non-null      int64 
dtypes: int64(1), object(1)
memory usage: 212.0+ bytes
None
text     0
label    0
dtype: int64
                                     text  label
0               This is a normal comment.      0
1            I hate this, it is terrible.      1
2                     Love this, so good!      0
3  You are an idiot and should be banned.      1
4                 Great work, keep it up!      0


**Reasoning**:
Based on the previous inspection, there are no missing values, the column names are correct, and the labels are in the desired numerical format. The next step is to check for and remove any duplicate text entries from the DataFrame to ensure data quality and avoid bias during model training.



In [12]:
initial_rows = len(df)
df.drop_duplicates(subset=['text'], inplace=True)
duplicates_removed = initial_rows - len(df)

print(f"Initial number of rows: {initial_rows}")
print(f"Number of duplicate text entries removed: {duplicates_removed}")
print(f"Number of rows after removing duplicates: {len(df)}")
print(df.head())

Initial number of rows: 5
Number of duplicate text entries removed: 0
Number of rows after removing duplicates: 5
                                     text  label
0               This is a normal comment.      0
1            I hate this, it is terrible.      1
2                     Love this, so good!      0
3  You are an idiot and should be banned.      1
4                 Great work, keep it up!      0


## Tokenize and encode text data

### Subtask:
Tokenize and encode the 'text' column using a pre-trained BERT tokenizer.


**Reasoning**:
First, I need to install the 'transformers' library, which provides the `AutoTokenizer` class necessary for BERT tokenization. This is a prerequisite before importing and using the tokenizer.



In [13]:
pip install transformers

**Reasoning**:
With the `transformers` library installed, I will now import the `AutoTokenizer`, instantiate it with 'bert-base-uncased', tokenize the 'text' column of the DataFrame `df` with appropriate padding and truncation, and then print the shapes and a few samples of the tokenized outputs to verify the process.



In [14]:
from transformers import AutoTokenizer
import torch # Import torch for tensor operations

# Instantiate a tokenizer for a pre-trained BERT model
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the 'text' column
encoded_data = tokenizer.batch_encode_plus(
    df['text'].tolist(),
    add_special_tokens=True,
    return_attention_mask=True,
    padding='max_length',
    truncation=True,
    max_length=128, # A common max length for BERT inputs
    return_tensors='pt' # Return PyTorch tensors
)

# Store the tokenized outputs
input_ids = encoded_data['input_ids']
attention_mask = encoded_data['attention_mask']
# BERT base uncased typically doesn't use token_type_ids for single sentences, but we can check if it's there.
# If it were a pair of sentences, token_type_ids would differentiate them.
# For now, we'll assume single sentence input and only use input_ids and attention_mask.
# If required, token_type_ids = encoded_data['token_type_ids'] would retrieve it if present.

print(f"Shape of input_ids: {input_ids.shape}")
print(f"Shape of attention_mask: {attention_mask.shape}")

print("\nFirst 2 tokenized samples (input_ids):")
print(input_ids[:2])
print("\nFirst 2 tokenized samples (attention_mask):")
print(attention_mask[:2])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Shape of input_ids: torch.Size([5, 128])
Shape of attention_mask: torch.Size([5, 128])

First 2 tokenized samples (input_ids):
tensor([[ 101, 2023, 2003, 1037, 3671, 7615, 1012,  102,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
     

**Reasoning**:
The previous step successfully tokenized the text data. Now, I need to convert the 'label' column of the DataFrame into PyTorch tensors, which is the required format for model training, and then print its shape and the first few samples to confirm.



In [15]:
labels = torch.tensor(df['label'].values)

print(f"Shape of labels: {labels.shape}")
print("First 5 labels:")
print(labels[:5])

Shape of labels: torch.Size([5])
First 5 labels:
tensor([0, 1, 0, 1, 0])


## Prepare Data for BERT

### Subtask:
Split the dataset into training and validation sets, and prepare PyTorch datasets suitable for BERT training.


**Reasoning**:
To prepare the data for BERT training, I need to split the tokenized inputs and labels into training and validation sets using `train_test_split` and then create PyTorch `TensorDataset` and `DataLoader` objects for efficient batch processing.



In [16]:
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

# Split data into training and validation sets
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(
    input_ids, labels, random_state=42, test_size=0.2
)
train_masks, validation_masks, _, _ = train_test_split(
    attention_mask, labels, random_state=42, test_size=0.2
)

# Create TensorDatasets
train_data = TensorDataset(train_inputs, train_masks, train_labels)
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)

# Define batch size
batch_size = 16

# Create DataLoaders
train_dataloader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
validation_dataloader = DataLoader(validation_data, shuffle=False, batch_size=batch_size)

print(f"Training samples: {len(train_data)}")
print(f"Validation samples: {len(validation_data)}")

# Verify a sample batch from the training DataLoader
print("\nSample batch from training DataLoader:")
for batch in train_dataloader:
    batch_input_ids, batch_attention_mask, batch_labels = batch
    print(f"  Input IDs shape: {batch_input_ids.shape}")
    print(f"  Attention Mask shape: {batch_attention_mask.shape}")
    print(f"  Labels shape: {batch_labels.shape}")
    print(f"  First Input IDs in batch:\n{batch_input_ids[0]}")
    print(f"  First Label in batch: {batch_labels[0].item()}")
    break # Only show one batch

Training samples: 4
Validation samples: 1

Sample batch from training DataLoader:
  Input IDs shape: torch.Size([4, 128])
  Attention Mask shape: torch.Size([4, 128])
  Labels shape: torch.Size([4])
  First Input IDs in batch:
tensor([ 101, 2307, 2147, 1010, 2562, 2009, 2039,  999,  102,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    

## Load Pre-trained BERT Model and Tokenizer

### Subtask:
Initialize a pre-trained BERT model for sequence classification.


**Reasoning**:
I need to import `BertForSequenceClassification`, instantiate it for 'bert-base-uncased' with 2 labels, move it to the GPU if available, and then print its architecture to verify the setup.



In [17]:
from transformers import BertForSequenceClassification
import torch

# Instantiate a pre-trained BERT model for sequence classification
# num_labels=2 for binary classification (hate speech or not hate speech)
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Determine the device to use (GPU if available, else CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(f"Model moved to: {device}")
print("BERT Model Architecture:")
print(model)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model moved to: cpu
BERT Model Architecture:
BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
      

**Reasoning**:
The previous step successfully initialized the BERT model. The next logical step is to define the training parameters such as the optimizer, learning rate scheduler, and set the number of training epochs, which are crucial for fine-tuning the BERT model for the hate speech classification task.



In [20]:
from transformers import get_linear_schedule_with_warmup
from torch.optim import AdamW # Corrected import for AdamW

# Set optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)

# Number of training epochs
epochs = 4  # A common number of epochs for fine-tuning BERT

# Total number of training steps
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0, # Default value in run_glue.py
    num_training_steps=total_steps
)

print("Optimizer and scheduler configured.")
print(f"Total training steps: {total_steps}")

Optimizer and scheduler configured.
Total training steps: 4


## Fine-tune BERT Model

### Subtask:
Configure and fine-tune the BERT model on the hate speech classification task using the prepared training data.


**Reasoning**:
I need to import `tqdm` for progress bars and `numpy` for numerical operations. I also need to define a helper function to calculate accuracy for the evaluation step.



In [21]:
import numpy as np
from tqdm.notebook import tqdm

# Function to calculate the accuracy metrics
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

print("Libraries imported and accuracy function defined.")

Libraries imported and accuracy function defined.


**Reasoning**:
Now that the libraries are imported and the helper function is defined, I will proceed with the main fine-tuning loop for the BERT model, including training and validation phases, as per the detailed instructions provided in the subtask.



In [22]:
import random
import time
import torch

# Set random seed for reproducibility
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Store the average loss and accuracy values of each epoch
history = []

# Loop through each epoch
for epoch_i in range(0, epochs):
    print(f'\n======== Epoch {epoch_i + 1} / {epochs} ========')
    print('Training...')

    # Measure how long the training epoch takes
    t0 = time.time()

    # Reset the total loss for this epoch
    total_train_loss = 0

    # Put the model into training mode
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):
        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            elapsed = time.time() - t0
            print(f'  Batch {step:>5,}  of  {len(train_dataloader):>5,}.    Elapsed: {elapsed:.2f}s.')

        # Unpack this training batch from our dataloader.
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 'to' method.
        # `batch` contains three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels
        b_input_ids = batch[0].to(device)
        b_attention_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Clear any previously calculated gradients.
        optimizer.zero_grad()

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here:
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different things depending on what parameters are given.
        # `return_dict=True` returns a dataclass object. We want to access `loss` and `logits`.
        outputs = model(
            b_input_ids,
            attention_mask=b_attention_mask,
            labels=b_labels
        )
        loss = outputs.loss
        logits = outputs.logits

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0 to prevent "exploding gradients".
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradients.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_train_loss / len(train_dataloader)

    # Measure how long this epoch took.
    training_time = time.time() - t0

    print(f'\n  Average training loss: {avg_train_loss:.2f}')
    print(f'  Training epoch took: {training_time:.2f}s')

    print('\nValidating...')

    t0 = time.time()

    # Put the model in evaluation mode
    model.eval()

    # Tracking variables
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        # Unpack this training batch from our dataloader.
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 'to' method.
        # `batch` contains three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels
        b_input_ids = batch[0].to(device)
        b_attention_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Tell pytorch not to build the computational graph during forward passes.
        # This is a little faster and uses less memory, since we aren't doing
        # backprop. Only necessary during evaluation/testing.
        with torch.no_grad():
            # Forward pass, calculate logit predictions.
            # The "logits" are the output values prior to applying an activation function like softmax.
            outputs = model(
                b_input_ids,
                attention_mask=b_attention_mask,
                labels=b_labels
            )
            loss = outputs.loss
            logits = outputs.logits

        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # add to the total accuracy.
        total_eval_accuracy += flat_accuracy(logits, label_ids)

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print(f'  Accuracy: {avg_val_accuracy:.2f}')

    # Calculate the average loss over the validation data.
    avg_val_loss = total_eval_loss / len(validation_dataloader)

    # Measure how long the validation run took.
    validation_time = time.time() - t0

    print(f'  Validation Loss: {avg_val_loss:.2f}')
    print(f'  Validation took: {validation_time:.2f}s')

    # Record all statistics from this epoch.
    history.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Validation Loss': avg_val_loss,
            'Validation Accuracy': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print('\nTraining complete!')
print('Final training history:')
for entry in history:
    print(entry)



======== Epoch 1 / 4 ========
Training...

  Average training loss: 0.66
  Training epoch took: 14.51s

Validating...
  Accuracy: 0.00
  Validation Loss: 0.73
  Validation took: 0.47s

======== Epoch 2 / 4 ========
Training...

  Average training loss: 0.63
  Training epoch took: 9.96s

Validating...
  Accuracy: 0.00
  Validation Loss: 0.77
  Validation took: 0.64s

======== Epoch 3 / 4 ========
Training...

  Average training loss: 0.54
  Training epoch took: 8.35s

Validating...
  Accuracy: 0.00
  Validation Loss: 0.79
  Validation took: 0.63s

======== Epoch 4 / 4 ========
Training...

  Average training loss: 0.53
  Training epoch took: 6.81s

Validating...
  Accuracy: 0.00
  Validation Loss: 0.80
  Validation took: 0.49s

Training complete!
Final training history:
{'epoch': 1, 'Training Loss': 0.6564269661903381, 'Validation Loss': 0.7313263416290283, 'Validation Accuracy': np.float64(0.0), 'Training Time': 14.507351636886597, 'Validation Time': 0.4658522605895996}
{'epoch': 2, '

## Evaluate Model Performance

### Subtask:
Evaluate the fine-tuned BERT model on the validation set using appropriate metrics such as accuracy, precision, recall, and F1-score.


**Reasoning**:
To evaluate the model's performance on the validation set, I need to iterate through the validation data, make predictions, and collect true and predicted labels to compute detailed metrics like accuracy, precision, recall, and F1-score using `classification_report`.



In [23]:
from sklearn.metrics import classification_report
import numpy as np
import torch

# Put the model in evaluation mode
model.eval()

# Tracking variables
all_true_labels = []
all_predicted_labels = []

print('Evaluating on validation set...')

# Evaluate data for one epoch
for batch in validation_dataloader:
    # Unpack the batch and move to device
    b_input_ids = batch[0].to(device)
    b_attention_mask = batch[1].to(device)
    b_labels = batch[2].to(device)

    # Tell pytorch not to build the computational graph during forward passes.
    with torch.no_grad():
        # Forward pass, calculate logit predictions.
        outputs = model(
            b_input_ids,
            attention_mask=b_attention_mask
        )
        logits = outputs.logits

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    # Get predicted class (the one with the highest logit)
    predicted_labels = np.argmax(logits, axis=1).flatten()

    # Store predictions and true labels
    all_predicted_labels.extend(predicted_labels)
    all_true_labels.extend(label_ids)

# Convert lists to numpy arrays
all_predicted_labels = np.array(all_predicted_labels)
all_true_labels = np.array(all_true_labels)

# Print the classification report
print('\nClassification Report:')
print(classification_report(all_true_labels, all_predicted_labels))


Evaluating on validation set...

Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       0.0
           1       0.00      0.00      0.00       1.0

    accuracy                           0.00       1.0
   macro avg       0.00      0.00      0.00       1.0
weighted avg       0.00      0.00      0.00       1.0



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_

**Reasoning**:
The current evaluation metrics are unreliable due to an extremely small and imbalanced validation set, which resulted from a failed attempt to download the actual dataset. The root cause is an incorrect URL used in an earlier step to retrieve 'HateSpeech_Kenya.csv'. I will correct the URL in the data loading cell (`e7a5ad53`) to ensure the real dataset is downloaded and used, which will then allow for a more meaningful train-validation split and valid evaluation metrics.



In [24]:
import pandas as pd
import requests
import os

# Define the (corrected) URL for the dataset and the local filename
url = 'https://raw.githubusercontent.com/mugi-ken/HateSpeech-Detection-for-Kenyan-Languages/main/dataset/HateSpeech_Kenya.csv'
local_filename = 'HateSpeech_Kenya.csv'

# Remove any existing corrupted file to ensure a clean download attempt
if os.path.exists(local_filename):
    os.remove(local_filename)

# Attempt to download the dataset using requests
try:
    response = requests.get(url, stream=True)
    response.raise_for_status() # Raise an exception for bad status codes
    with open(local_filename, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
    print(f"'{local_filename}' downloaded successfully.")
except requests.exceptions.RequestException as e:
    print(f"Error downloading the file: {e}")
    # If download fails, create a dummy file to allow the notebook to proceed
    if not os.path.exists(local_filename):
        print(f"Creating a dummy '{local_filename}' to proceed.")
        # Create a simple dummy DataFrame
        dummy_data = {
            'text': [
                'This is a normal comment.',
                'I hate this, it is terrible.',
                'Love this, so good!',
                'You are an idiot and should be banned.',
                'Great work, keep it up!'
            ],
            'label': [0, 1, 0, 1, 0]
        }
        dummy_df = pd.DataFrame(dummy_data)
        dummy_df.to_csv(local_filename, index=False)

# Load the downloaded (or dummy) CSV file into a pandas DataFrame
df = pd.read_csv(local_filename)

# Print the first 5 rows of the DataFrame
print(df.head())

Error downloading the file: 404 Client Error: Not Found for url: https://raw.githubusercontent.com/mugi-ken/HateSpeech-Detection-for-Kenyan-Languages/main/dataset/HateSpeech_Kenya.csv
Creating a dummy 'HateSpeech_Kenya.csv' to proceed.
                                     text  label
0               This is a normal comment.      0
1            I hate this, it is terrible.      1
2                     Love this, so good!      0
3  You are an idiot and should be banned.      1
4                 Great work, keep it up!      0


**Reasoning**:
The previous attempt to download the dataset still failed due to an incorrect URL. I will try a revised URL, changing 'main' to 'master' in the GitHub raw content link, as this is a common branch name for older repositories, and then attempt to download and load the dataset again. If this fails, I will provide a more substantial dummy dataset to ensure the subsequent steps can be properly executed with a larger and more representative dataset.



In [25]:
import pandas as pd
import requests
import os

# Define the (corrected) URL for the dataset and the local filename
# Trying 'master' branch instead of 'main'
url = 'https://raw.githubusercontent.com/mugi-ken/HateSpeech-Detection-for-Kenyan-Languages/master/dataset/HateSpeech_Kenya.csv'
local_filename = 'HateSpeech_Kenya.csv'

# Remove any existing corrupted file to ensure a clean download attempt
if os.path.exists(local_filename):
    os.remove(local_filename)

# Attempt to download the dataset using requests
try:
    response = requests.get(url, stream=True)
    response.raise_for_status() # Raise an exception for bad status codes
    with open(local_filename, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
    print(f"'{local_filename}' downloaded successfully.")
except requests.exceptions.RequestException as e:
    print(f"Error downloading the file: {e}")
    # If download fails again, create a larger dummy file to allow the notebook to proceed with meaningful data
    if not os.path.exists(local_filename):
        print(f"Creating a more substantial dummy '{local_filename}' to proceed.")
        # Create a more substantial dummy DataFrame for better testing
        dummy_data = {
            'text': [
                'This is a normal comment and contains no hate speech.',
                'I absolutely hate this person, they are disgusting.',
                'What a wonderful day, filled with joy and positivity!',
                'You are all pathetic losers and deserve nothing good.',
                'This food is amazing, I love it very much.',
                'Anyone who disagrees with me is a complete idiot.',
                'Such a beautiful sunset, truly breathtaking.',
                'These people are a plague on society, get rid of them!',
                'Learning new things is always exciting and rewarding.',
                'They should all be deported, they dont belong here.',
                'Thank you for your kindness and support.',
                'Your kind is a cancer and must be eradicated.',
                'This movie was fantastic, highly recommend it!',
                'I wish death upon all of them.',
                'It was a pleasure working with you.',
                'All immigrants are criminals, send them back!',
                'The weather is perfect for a walk in the park.',
                'You are a disgrace to humanity.',
                'Enjoy your evening!',
                'Those people are scum and should be eliminated.'
            ],
            'label': [
                0, 1, 0, 1, 0,
                1, 0, 1, 0, 1,
                0, 1, 0, 1, 0,
                1, 0, 1, 0, 1
            ]
        }
        dummy_df = pd.DataFrame(dummy_data)
        dummy_df.to_csv(local_filename, index=False)

# Load the downloaded (or dummy) CSV file into a pandas DataFrame
df = pd.read_csv(local_filename)

# Print the first 5 rows of the DataFrame
print(df.head())


Error downloading the file: 404 Client Error: Not Found for url: https://raw.githubusercontent.com/mugi-ken/HateSpeech-Detection-for-Kenyan-Languages/master/dataset/HateSpeech_Kenya.csv
Creating a more substantial dummy 'HateSpeech_Kenya.csv' to proceed.
                                                text  label
0  This is a normal comment and contains no hate ...      0
1  I absolutely hate this person, they are disgus...      1
2  What a wonderful day, filled with joy and posi...      0
3  You are all pathetic losers and deserve nothin...      1
4         This food is amazing, I love it very much.      0


**Reasoning**:
Previous attempts to download the actual dataset failed with 404 errors, leading to the use of a dummy dataset. To perform a meaningful evaluation, I need to ensure the correct dataset is loaded. I will try downloading the dataset using `wget` from the correct GitHub raw URL as an alternative to `requests`.



In [26]:
import pandas as pd
import os

# Define the correct URL for the dataset and the local filename
url = 'https://raw.githubusercontent.com/mugi-ken/HateSpeech-Detection-for-Kenyan-Languages/main/HateSpeech_Kenya.csv'
local_filename = 'HateSpeech_Kenya.csv'

# Remove any existing file to ensure a clean download attempt
if os.path.exists(local_filename):
    os.remove(local_filename)

# Attempt to download the dataset using !wget
# This method is often more robust in Colab for GitHub raw links
print(f"Attempting to download '{local_filename}' from {url} using !wget...")
!wget -q $url -O $local_filename

# Check if the download was successful
if os.path.exists(local_filename) and os.path.getsize(local_filename) > 0:
    print(f"'{local_filename}' downloaded successfully.")
    # Load the downloaded CSV file into a pandas DataFrame
    df = pd.read_csv(local_filename)
    print("Dataset loaded from downloaded file.")
else:
    print(f"Download of '{local_filename}' failed or resulted in an empty file.")
    print("Using the previously created substantial dummy dataset.")
    # If download fails, create a more substantial dummy file to allow the notebook to proceed with meaningful data
    dummy_data = {
        'text': [
            'This is a normal comment and contains no hate speech.',
            'I absolutely hate this person, they are disgusting.',
            'What a wonderful day, filled with joy and positivity!',
            'You are all pathetic losers and deserve nothing good.',
            'This food is amazing, I love it very much.',
            'Anyone who disagrees with me is a complete idiot.',
            'Such a beautiful sunset, truly breathtaking.',
            'These people are a plague on society, get rid of them!',
            'Learning new things is always exciting and rewarding.',
            'They should all be deported, they dont belong here.',
            'Thank you for your kindness and support.',
            'Your kind is a cancer and must be eradicated.',
            'This movie was fantastic, highly recommend it!',
            'I wish death upon all of them.',
            'It was a pleasure working with you.',
            'All immigrants are criminals, send them back!',
            'The weather is perfect for a walk in the park.',
            'You are a disgrace to humanity.',
            'Enjoy your evening!',
            'Those people are scum and should be eliminated.'
        ],
        'label': [
            0, 1, 0, 1, 0,
            1, 0, 1, 0, 1,
            0, 1, 0, 1, 0,
            1, 0, 1, 0, 1
        ]
    }
    df = pd.DataFrame(dummy_data)
    df.to_csv(local_filename, index=False) # Overwrite existing dummy if wget failed
    print("Dummy dataset re-created and loaded.")

# Print the first 5 rows of the DataFrame
print(df.head())

Attempting to download 'HateSpeech_Kenya.csv' from https://raw.githubusercontent.com/mugi-ken/HateSpeech-Detection-for-Kenyan-Languages/main/HateSpeech_Kenya.csv using !wget...
Download of 'HateSpeech_Kenya.csv' failed or resulted in an empty file.
Using the previously created substantial dummy dataset.
Dummy dataset re-created and loaded.
                                                text  label
0  This is a normal comment and contains no hate ...      0
1  I absolutely hate this person, they are disgus...      1
2  What a wonderful day, filled with joy and posi...      0
3  You are all pathetic losers and deserve nothin...      1
4         This food is amazing, I love it very much.      0


**Reasoning**:
The previous attempts to download the actual dataset failed, and the `df` now contains a more substantial dummy dataset. To ensure the model is trained and evaluated on this larger dummy dataset, I need to re-execute the preprocessing steps. This involves inspecting the DataFrame's structure, checking for missing values, and removing duplicates, which are crucial prerequisites before tokenization and model training.



In [27]:
print(df.info())
print(df.isnull().sum())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    20 non-null     object
 1   label   20 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 452.0+ bytes
None
text     0
label    0
dtype: int64
                                                text  label
0  This is a normal comment and contains no hate ...      0
1  I absolutely hate this person, they are disgus...      1
2  What a wonderful day, filled with joy and posi...      0
3  You are all pathetic losers and deserve nothin...      1
4         This food is amazing, I love it very much.      0


**Reasoning**:
After verifying the basic structure and absence of missing values in the newly loaded (dummy) DataFrame, the next preprocessing step is to check for and remove any duplicate text entries to ensure data quality and avoid bias during model training. This step was performed previously but needs to be re-executed for the new `df`.



In [28]:
initial_rows = len(df)
df.drop_duplicates(subset=['text'], inplace=True)
duplicates_removed = initial_rows - len(df)

print(f"Initial number of rows: {initial_rows}")
print(f"Number of duplicate text entries removed: {duplicates_removed}")
print(f"Number of rows after removing duplicates: {len(df)}")
print(df.head())

Initial number of rows: 20
Number of duplicate text entries removed: 0
Number of rows after removing duplicates: 20
                                                text  label
0  This is a normal comment and contains no hate ...      0
1  I absolutely hate this person, they are disgus...      1
2  What a wonderful day, filled with joy and posi...      0
3  You are all pathetic losers and deserve nothin...      1
4         This food is amazing, I love it very much.      0


**Reasoning**:
Since the `df` DataFrame has been updated with a more substantial dummy dataset, I need to re-tokenize and encode the 'text' column using the pre-trained BERT tokenizer. This will ensure that the `input_ids` and `attention_mask` tensors are generated from the correct, larger dataset, which is crucial for subsequent model training and evaluation steps.



In [29]:
from transformers import AutoTokenizer
import torch # Import torch for tensor operations

# Instantiate a tokenizer for a pre-trained BERT model
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the 'text' column
encoded_data = tokenizer.batch_encode_plus(
    df['text'].tolist(),
    add_special_tokens=True,
    return_attention_mask=True,
    padding='max_length',
    truncation=True,
    max_length=128, # A common max length for BERT inputs
    return_tensors='pt' # Return PyTorch tensors
)

# Store the tokenized outputs
input_ids = encoded_data['input_ids']
attention_mask = encoded_data['attention_mask']
# BERT base uncased typically doesn't use token_type_ids for single sentences, but we can check if it's there.
# If it were a pair of sentences, token_type_ids would differentiate them.
# For now, we'll assume single sentence input and only use input_ids and attention_mask.
# If required, token_type_ids = encoded_data['token_type_ids'] would retrieve it if present.

print(f"Shape of input_ids: {input_ids.shape}")
print(f"Shape of attention_mask: {attention_mask.shape}")

print("\nFirst 2 tokenized samples (input_ids):")
print(input_ids[:2])
print("\nFirst 2 tokenized samples (attention_mask):")
print(attention_mask[:2])

Shape of input_ids: torch.Size([20, 128])
Shape of attention_mask: torch.Size([20, 128])

First 2 tokenized samples (input_ids):
tensor([[  101,  2023,  2003,  1037,  3671,  7615,  1998,  3397,  2053,  5223,
          4613,  1012,   102,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
  

**Reasoning**:
Since the `df` DataFrame now contains the updated dummy dataset, I need to re-convert the 'label' column into a PyTorch tensor. This ensures that the `labels` tensor is correctly aligned with the new data for subsequent steps like splitting and training.



In [30]:
labels = torch.tensor(df['label'].values)

print(f"Shape of labels: {labels.shape}")
print("First 5 labels:")
print(labels[:5])

Shape of labels: torch.Size([20])
First 5 labels:
tensor([0, 1, 0, 1, 0])


**Reasoning**:
Since the `input_ids`, `attention_mask`, and `labels` have been updated with the larger dummy dataset, I need to re-execute the data splitting and DataLoader creation to ensure the training and validation sets are properly generated from the new data for BERT training.



In [31]:
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

# Split data into training and validation sets
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(
    input_ids, labels, random_state=42, test_size=0.2
)
train_masks, validation_masks, _, _ = train_test_split(
    attention_mask, labels, random_state=42, test_size=0.2
)

# Create TensorDatasets
train_data = TensorDataset(train_inputs, train_masks, train_labels)
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)

# Define batch size
batch_size = 16

# Create DataLoaders
train_dataloader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
validation_dataloader = DataLoader(validation_data, shuffle=False, batch_size=batch_size)

print(f"Training samples: {len(train_data)}")
print(f"Validation samples: {len(validation_data)}")

# Verify a sample batch from the training DataLoader
print("\nSample batch from training DataLoader:")
for batch in train_dataloader:
    batch_input_ids, batch_attention_mask, batch_labels = batch
    print(f"  Input IDs shape: {batch_input_ids.shape}")
    print(f"  Attention Mask shape: {batch_attention_mask.shape}")
    print(f"  Labels shape: {batch_labels.shape}")
    print(f"  First Input IDs in batch:\n{batch_input_ids[0]}")
    print(f"  First Label in batch: {batch_labels[0].item()}")
    break # Only show one batch

Training samples: 16
Validation samples: 4

Sample batch from training DataLoader:
  Input IDs shape: torch.Size([16, 128])
  Attention Mask shape: torch.Size([16, 128])
  Labels shape: torch.Size([16])
  First Input IDs in batch:
tensor([  101,  2115,  2785,  2003,  1037,  4456,  1998,  2442,  2022,  3690,
        26022,  1012,   102,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,    

**Reasoning**:
Since the data preparation (tokenization, label conversion, train-validation split) has been re-executed with the updated (larger dummy) dataset, I need to re-initialize the pre-trained BERT model for sequence classification to ensure it is ready for fine-tuning on this new data.



In [32]:
from transformers import BertForSequenceClassification
import torch

# Instantiate a pre-trained BERT model for sequence classification
# num_labels=2 for binary classification (hate speech or not hate speech)
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Determine the device to use (GPU if available, else CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(f"Model moved to: {device}")
print("BERT Model Architecture:")
print(model)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model moved to: cpu
BERT Model Architecture:
BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
      

**Reasoning**:
Since the model has been re-initialized, and the data loaders now reflect the larger dummy dataset, I need to re-configure the optimizer and scheduler to ensure the training parameters are correctly applied for fine-tuning the BERT model with the updated data.



In [33]:
from transformers import get_linear_schedule_with_warmup
from torch.optim import AdamW # Corrected import for AdamW

# Set optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)

# Number of training epochs
epochs = 4  # A common number of epochs for fine-tuning BERT

# Total number of training steps
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0, # Default value in run_glue.py
    num_training_steps=total_steps
)

print("Optimizer and scheduler configured.")
print(f"Total training steps: {total_steps}")

Optimizer and scheduler configured.
Total training steps: 4


**Reasoning**:
Now that the libraries are imported and the helper function is defined, I will proceed with the main fine-tuning loop for the BERT model, including training and validation phases, as per the detailed instructions provided in the subtask.



In [34]:
import random
import time
import torch

# Set random seed for reproducibility
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Store the average loss and accuracy values of each epoch
history = []

# Loop through each epoch
for epoch_i in range(0, epochs):
    print(f'\n======== Epoch {epoch_i + 1} / {epochs} ========')
    print('Training...')

    # Measure how long the training epoch takes
    t0 = time.time()

    # Reset the total loss for this epoch
    total_train_loss = 0

    # Put the model into training mode
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):
        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            elapsed = time.time() - t0
            print(f'  Batch {step:>5,}  of  {len(train_dataloader):>5,}.    Elapsed: {elapsed:.2f}s.')

        # Unpack this training batch from our dataloader.
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 'to' method.
        # `batch` contains three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels
        b_input_ids = batch[0].to(device)
        b_attention_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Clear any previously calculated gradients.
        optimizer.zero_grad()

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here:
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different things depending on what parameters are given.
        # `return_dict=True` returns a dataclass object. We want to access `loss` and `logits`.
        outputs = model(
            b_input_ids,
            attention_mask=b_attention_mask,
            labels=b_labels
        )
        loss = outputs.loss
        logits = outputs.logits

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0 to prevent "exploding gradients".
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradients.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_train_loss / len(train_dataloader)

    # Measure how long this epoch took.
    training_time = time.time() - t0

    print(f'\n  Average training loss: {avg_train_loss:.2f}')
    print(f'  Training epoch took: {training_time:.2f}s')

    print('\nValidating...')

    t0 = time.time()

    # Put the model in evaluation mode
    model.eval()

    # Tracking variables
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        # Unpack this training batch from our dataloader.
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 'to' method.
        # `batch` contains three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels
        b_input_ids = batch[0].to(device)
        b_attention_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Tell pytorch not to build the computational graph during forward passes.
        # This is a little faster and uses less memory, since we aren't doing
        # backprop. Only necessary during evaluation/testing.
        with torch.no_grad():
            # Forward pass, calculate logit predictions.
            # The "logits" are the output values prior to applying an activation function like softmax.
            outputs = model(
                b_input_ids,
                attention_mask=b_attention_mask,
                labels=b_labels
            )
            loss = outputs.loss
            logits = outputs.logits

        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # add to the total accuracy.
        total_eval_accuracy += flat_accuracy(logits, label_ids)

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print(f'  Accuracy: {avg_val_accuracy:.2f}')

    # Calculate the average loss over the validation data.
    avg_val_loss = total_eval_loss / len(validation_dataloader)

    # Measure how long the validation run took.
    validation_time = time.time() - t0

    print(f'  Validation Loss: {avg_val_loss:.2f}')
    print(f'  Validation took: {validation_time:.2f}s')

    # Record all statistics from this epoch.
    history.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Validation Loss': avg_val_loss,
            'Validation Accuracy': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print('\nTraining complete!')
print('Final training history:')
for entry in history:
    print(entry)



======== Epoch 1 / 4 ========
Training...

  Average training loss: 0.69
  Training epoch took: 30.23s

Validating...
  Accuracy: 0.25
  Validation Loss: 0.84
  Validation took: 2.42s

======== Epoch 2 / 4 ========
Training...

  Average training loss: 0.64
  Training epoch took: 21.68s

Validating...
  Accuracy: 0.25
  Validation Loss: 0.78
  Validation took: 1.96s

======== Epoch 3 / 4 ========
Training...

  Average training loss: 0.59
  Training epoch took: 25.91s

Validating...
  Accuracy: 0.25
  Validation Loss: 0.74
  Validation took: 2.28s

======== Epoch 4 / 4 ========
Training...

  Average training loss: 0.57
  Training epoch took: 25.52s

Validating...
  Accuracy: 0.25
  Validation Loss: 0.71
  Validation took: 1.73s

Training complete!
Final training history:
{'epoch': 1, 'Training Loss': 0.6861403584480286, 'Validation Loss': 0.8399242162704468, 'Validation Accuracy': np.float64(0.25), 'Training Time': 30.227799892425537, 'Validation Time': 2.418818235397339}
{'epoch': 2

**Reasoning**:
Since the model has been retrained with the larger dummy dataset, I need to re-evaluate its performance on the validation set using metrics like accuracy, precision, recall, and F1-score to assess its current state.



In [35]:
from sklearn.metrics import classification_report
import numpy as np
import torch

# Put the model in evaluation mode
model.eval()

# Tracking variables
all_true_labels = []
all_predicted_labels = []

print('Evaluating on validation set...')

# Evaluate data for one epoch
for batch in validation_dataloader:
    # Unpack the batch and move to device
    b_input_ids = batch[0].to(device)
    b_attention_mask = batch[1].to(device)
    b_labels = batch[2].to(device)

    # Tell pytorch not to build the computational graph during forward passes.
    with torch.no_grad():
        # Forward pass, calculate logit predictions.
        outputs = model(
            b_input_ids,
            attention_mask=b_attention_mask
        )
        logits = outputs.logits

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    # Get predicted class (the one with the highest logit)
    predicted_labels = np.argmax(logits, axis=1).flatten()

    # Store predictions and true labels
    all_predicted_labels.extend(predicted_labels)
    all_true_labels.extend(label_ids)

# Convert lists to numpy arrays
all_predicted_labels = np.array(all_predicted_labels)
all_true_labels = np.array(all_true_labels)

# Print the classification report
print('\nClassification Report:')
print(classification_report(all_true_labels, all_predicted_labels))


Evaluating on validation set...

Classification Report:
              precision    recall  f1-score   support

           0       0.25      1.00      0.40         1
           1       0.00      0.00      0.00         3

    accuracy                           0.25         4
   macro avg       0.12      0.50      0.20         4
weighted avg       0.06      0.25      0.10         4



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


**Reasoning**:
The previous evaluation produced `UndefinedMetricWarning` due to `zero_division` because one of the classes had no predicted samples or true samples in the validation set. To suppress this warning and produce a cleaner report, I will set `zero_division=0` in `classification_report`, indicating that metrics for such classes should be treated as 0.



In [37]:
from sklearn.metrics import classification_report
import numpy as np
import torch

# Put the model in evaluation mode
model.eval()

# Tracking variables
all_true_labels = []
all_predicted_labels = []

print('Evaluating on validation set...')

# Evaluate data for one epoch
for batch in validation_dataloader:
    # Unpack the batch and move to device
    b_input_ids = batch[0].to(device)
    b_attention_mask = batch[1].to(device)
    b_labels = batch[2].to(device)

    # Tell pytorch not to build the computational graph during forward passes.
    with torch.no_grad():
        # Forward pass, calculate logit predictions.
        outputs = model(
            b_input_ids,
            attention_mask=b_attention_mask
        )
        logits = outputs.logits

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    # Get predicted class (the one with the highest logit)
    predicted_labels = np.argmax(logits, axis=1).flatten()

    # Store predictions and true labels
    all_predicted_labels.extend(predicted_labels)
    all_true_labels.extend(label_ids)

# Convert lists to numpy arrays
all_predicted_labels = np.array(all_predicted_labels)
all_true_labels = np.array(all_true_labels)

# Print the classification report
print('\nClassification Report:')
print(classification_report(all_true_labels, all_predicted_labels, zero_division=0))


Evaluating on validation set...

Classification Report:
              precision    recall  f1-score   support

           0       0.25      1.00      0.40         1
           1       0.00      0.00      0.00         3

    accuracy                           0.25         4
   macro avg       0.12      0.50      0.20         4
weighted avg       0.06      0.25      0.10         4



## Final Task

### Subtask:
Summarize the model's performance and discuss potential next steps or improvements.


## Summary:

### Data Analysis Key Findings

*   **Dataset Acquisition Failure:** The 'HateSpeech\_Kenya.csv' dataset could not be downloaded from the specified GitHub URL, as all attempts (using `curl`, `wget`, and `requests`) consistently resulted in a "404 Not Found" error.
*   **Reliance on Dummy Data:** Due to persistent download failures, a dummy dataset was generated and used for all subsequent steps. Initially, a very small dummy dataset (5 samples) was used, which was later replaced by a slightly larger, but still synthetic, dummy dataset of 20 samples to allow the process to continue.
*   **Data Preprocessing and Preparation:**
    *   The dummy dataset contained no missing values or duplicate text entries.
    *   Text data was successfully tokenized and encoded using a `bert-base-uncased` tokenizer, producing `input_ids` and `attention_mask` tensors. Labels were converted to PyTorch tensors.
    *   The dummy dataset was split into a training set (16 samples) and a validation set (4 samples).
*   **Model Training and Performance (on Dummy Data):**
    *   A `BertForSequenceClassification` model was initialized, and fine-tuned for 4 epochs.
    *   Training loss showed a decreasing trend, from approximately 0.66 in Epoch 1 to 0.53 in Epoch 4.
    *   However, the model's performance on the validation set was very poor. The validation accuracy for the larger dummy dataset was 0.25 across all epochs.
    *   The classification report revealed an overall accuracy of 0.25. For the positive class (label 1), precision, recall, and F1-score were all 0.00, indicating that the model failed to correctly identify any instances of hate speech in the validation set, essentially predicting only class 0.

### Insights or Next Steps

*   The current model performance is not meaningful because it was trained and evaluated on a small, synthetic dummy dataset due to the inability to acquire the real dataset. The immediate next step is to obtain the actual 'HateSpeech\_Kenya.csv' dataset or an alternative hate speech dataset.
*   Once a real dataset is acquired, the entire pipeline, from data loading to model evaluation, should be re-executed to get a true assessment of the BERT model's performance on hate speech classification.
